import libraries and read data

In [ ]:
!pip uninstall eodal -y
!pip install git+https://github.com/lukasValentin/eodal.git@landsat-dev

In [4]:
#import what is needed
from datetime import datetime
from pathlib import Path
from shapely.geometry import box

from eodal.config import get_settings
from eodal.core.sensors import Landsat
from eodal.core.scene import SceneCollection
from eodal.mapper.feature import Feature

In [5]:
# read data from pickled file object into SceneCollectio
fpath = Path('Y:\MSc_23_TimckeFinn\data\EOdal\sample_mapper_data.pkl')
scoll = SceneCollection.from_pickle(stream=fpath)
scoll

EOdal SceneCollection
----------------------
# Scenes:    1
Timestamps:    2023-05-14 00:00:00
Scene Identifiers:    LC09_L2SP_173060_20230514_20230517_02_T1

delete no data

In [67]:
# loop over available scenes stored in scoll as a EOdal SceneCollection and check
# for no-data. These scenes will be removed from the SceneCollection
scenes_to_del = []
scoll.metadata['scene_used'] = 'yes'

for scene_id, scene in scoll:

    # check if scene is blackfilled (nodata); if yes continue
    if scene.is_blackfilled:
        scenes_to_del.append(scene_id)
        mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [blackfill]'
        continue

    # check cloud coverage (including shadows and snow) of the field parcel
    feature_cloud_cover = scene.get_cloudy_pixel_percentage(cloud_classes=[3, 8, 9, 10, 11])

    # if the scene is too cloudy, we skip it
    if feature_cloud_cover > feature_cloud_cover_threshold:
        scenes_to_del.append(scene_id)
        mapper.metadata.loc[mapper.metadata.sensing_time.dt.strftime('%Y-%m-%d %H:%M') == scene_id.strftime('%Y-%m-%d %H:%M')[0:16], 'scene_used'] = 'No [clouds]'
        continue

    # calculate the NDVI and MSAVI
    scene.calc_si('NDVI', inplace=True)
    scene.calc_si('MSAVI', inplace=True)

# delete scenes too cloudy or containing only no-data
for scene_id in scenes_to_del:
    del mapper.data[scene_id]

AttributeError: 'SceneCollection' object has no attribute 'metadata'

Get values

In [7]:
for scene_id, scene in scoll:
   x = scene.get_values()
   
x


masked_array(
  data=[[[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        [[--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --]],

        ...,

        [[--, --, --, ..., 1.0, 1.0, 1.0],
         [--, --, --, ..., 1.0, 1.0, 1.0],
         [--, --, --, ..., 1.0, 1.0, 1.0],
         ...,
         [--, --, --, ..., 1.0, 1.0, 1.0],
         [--, --, 